Qual o objetivo do comando cache​ ​em Spark?

O mesmo código implementado em Spark é normalmente mais rápido que a implementação equivalente em
MapReduce. Por quê?

Por causa da sua arquitetura que roda grande parte do processamento na memoria RAM. E suas tarefas apenas executam quando a operação reduce é chamada.

Qual é a função do SparkContext​?

Define a conexão com um cluster do Apache Spark.

Explique com suas palavras o que é Resilient​ ​Distributed​ ​Datasets​ (RDD).


Uma coleção de objetos que são distribuidos em um ou mais partições da memoria. Dessa forma os dados podem ser tratados de forma paralela 

GroupByKey​ ​é menos eficiente que reduceByKey​ ​em grandes dataset. Por quê?

GroupByKey não é realizado de forma paralela, dessa forma acaba por com todo o principio de clusterização do Apache

Explique o que o código Scala abaixo faz:
val textFile = sc.textFile("hdfs://...")
val counts = textFile.flatMap(line => line.split(" "))
.map(word => (word, 1))
.reduceByKey(_ + _)
counts.saveAsTextFile("hdfs://...")


Carrega o arquivo e conta a quantida de pelavras dentro desse arquivo.

# NASA-HTTP

In [14]:
from pyspark.sql import SQLContext
from pyspark import SparkContext
sc = SparkContext()
sqlContext = SQLContext(sc)

# Carregando os arquivos

In [20]:
arquivo1 = sqlContext.read.text( "data/access_log_Aug95")
arquivo2 = sqlContext.read.text( "data/access_log_Jul95")

log_file = arquivo1.union(arquivo2) 
log_file.show(5, truncate=False)

+--------------------------------------------------------------------------------------------------------------------------+
|value                                                                                                                     |
+--------------------------------------------------------------------------------------------------------------------------+
|in24.inetnebr.com - - [01/Aug/1995:00:00:01 -0400] "GET /shuttle/missions/sts-68/news/sts-68-mcc-05.txt HTTP/1.0" 200 1839|
|uplherc.upl.com - - [01/Aug/1995:00:00:07 -0400] "GET / HTTP/1.0" 304 0                                                   |
|uplherc.upl.com - - [01/Aug/1995:00:00:08 -0400] "GET /images/ksclogo-medium.gif HTTP/1.0" 304 0                          |
|uplherc.upl.com - - [01/Aug/1995:00:00:08 -0400] "GET /images/MOSAIC-logosmall.gif HTTP/1.0" 304 0                        |
|uplherc.upl.com - - [01/Aug/1995:00:00:08 -0400] "GET /images/USA-logosmall.gif HTTP/1.0" 304 0                           |


In [26]:
import pyspark.sql.functions
split_df = log_file.select(regexp_extract('value', r'^([^\s]+\s)', 1).alias('host'),
                        regexp_extract('value', r'^.*\[(\d\d/\w{3}/\d{4}:\d{2}:\d{2}:\d{2} -\d{4})]', 1).alias('timestamp'),
                        regexp_extract('value', r'^.*"\w+\s+([^\s]+)\s+HTTP.*"', 1).alias('path'),
                        regexp_extract('value', r'^.*"\s+([^\s]+)', 1).cast('integer').alias('status'),
                        regexp_extract('value', r'^.*\s+(\d+)$', 1).cast('integer').alias('content_size'))
split_df.show(2, truncate=False)

+------------------+--------------------------+-----------------------------------------------+------+------------+
|host              |timestamp                 |path                                           |status|content_size|
+------------------+--------------------------+-----------------------------------------------+------+------------+
|in24.inetnebr.com |01/Aug/1995:00:00:01 -0400|/shuttle/missions/sts-68/news/sts-68-mcc-05.txt|200   |1839        |
|uplherc.upl.com   |01/Aug/1995:00:00:07 -0400|/                                              |304   |0           |
+------------------+--------------------------+-----------------------------------------------+------+------------+
only showing top 2 rows



### Quantidade de hosts

In [55]:
unique_hosts = split_df.select('host').distinct().count()
print("Hosts únicos:" + unique_hosts)

137979


### Quantidade de retornos 404

In [35]:
invalid_response = split_df.filter(split_df.status.contains("404")).count()
print("Total de 404:" + invalid_response)

20901


### Maiores causadores de erro

In [44]:
group_404 = split_df.filter('status=404')

In [49]:
most_404 = group_404.select('host').groupBy('host').count().orderBy('count', ascending=False)
most_404.show(5, truncate = False)

+----------------------------+-----+
|host                        |count|
+----------------------------+-----+
|hoohoo.ncsa.uiuc.edu        |251  |
|piweba3y.prodigy.com        |157  |
|jbiagioni.npt.nuwc.navy.mil |132  |
|piweba1y.prodigy.com        |114  |
|www-d4.proxy.aol.com        |91   |
+----------------------------+-----+
only showing top 5 rows



### Bytes retornados